# Ujian Akhir Semester
# Analitika Media Sosial
# A12.6501

## Nama : Agung Kurniawan
## NIM : A12.2020.06364

## Movie Recommender System, menggunakan metode Collaborative Filtering

Poin pertama pada laporan ini adalah bagaimana proses movie recomender system dijalankan. langkah pertama yang dijalankan yaitu import library Pandas, proses tersebut menggunakan metode collaborative filtering.
    Sebelumnya saya telah mencari dataset barang dari Pt Indomarco Prismatama namun tidak menemukannya dikarenakan data tersebut bersifat privat sehingga saya memutuskan untuk merubah dataset yang akan saya gunakan. saya mencari dataset dari website https://www.kaggle.com/code/indralin/movielens-project-1-1-content-based/data?select=movie.csv disitu tedapat dataset yang saya butuhkan seperti dataset movie dan rating.


### Collaborative Filtering

Collaborative filtering adalah salah satu metode yang digunakan dalam data analis untuk mengidentifikasi preferensi atau kecenderungan seseorang terhadap suatu item (item disini dapat diartikan sebagai film) berdasarkan pada input dari orang lain yang memiliki preferensi yang sama atau mirip. Dalam metode collaborative filtering, sistem menggunakan informasi tentang bagaimana orang lain yang memiliki preferensi yang sama atau mirip dengan pengguna tersebut telah memberikan rating atau evaluasi terhadap item-item tertentu. Dengan demikian, metode collaborative filtering berfokus pada kolaborasi antara pengguna dalam memberikan rekomendasi.

### Library Pandas

Pandas adalah library Python yang digunakan untuk mengolah data. alasan saya menggunakan library pandas yakni karena library ini sangat berguna pada proses pengolahan data yang bersifat tabular, seperti data yang disimpan dalam bentuk CSV atau spreadsheet. 

Pandas juga menawarkan berbagai fitur seperti:
- Melakukan cleaning data, seperti menghapus baris atau kolom yang tidak diperlukan, mengisi data yang kosong, dan mengubah tipe data.
- Melakukan manipulasi data, seperti menghitung statistik deskriptif, melakukan agregasi data, dan mengelompokkan data berdasarkan kategori.
- Melakukan visualisasi data, seperti membuat grafik dan plot.

### Import Library

In [1]:
import pandas as pd
pd.set_option( 'display.max_columns', None)
pd.set_option( 'display.max_rows', None)

movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000797 entries, 0 to 20000796
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     float64
 4   rating     float64
 5   timestamp  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.0+ GB


## Data Preparation

### Menghilangkan simbol pada judul untuk mencegah rusaknya data

In [2]:
df['year_movie'] = df.title.str.extract('(\(\d\d\d\d\))',expand=False)
df['year_movie'] = df.year_movie.str.extract('(\d\d\d\d)',expand=False)

### Menghapus tahun pada kolom "title"

In [3]:
df['title']= df.title.str.replace ('(\(\d\d\d\d\))', '',regex=True)

### Menghilangkan spasi pada akhir judul

In [4]:
df['title']= df['title'].apply(lambda x: x.strip())

- proses diatas menghilangkan tanda kurung, menghapus tahun dan space pada kolom judul. bertujuan untuk mencegah rusaknya data pada saat proses selanjutnya

### Mengkategorikan film berdasarkan tingkat kepopuleran

In [5]:
a = pd.DataFrame(df["title"].value_counts())
rare_movies = a[a["title"]<= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

* berikutnya mengkategorikan film berdasarkan jumlah yang di tonton atau tingkat kepopuleran. 
* Disini terdapat 2 kategori yakni rare movies dan common movies. rare movies yaitu film yang ditonton kurang dari sama dengan 1000 kali sedangkan common movies yaitu film yang diluar kategori rare movies.

### Setiap genre dipisahkan oleh tanda "|" 

In [6]:
df["genre"] = df["genres"].apply(lambda x: x.split("|")[0])
df.drop("genres", inplace=True, axis=1)
df.head()

,movieId,title,userId,rating,timestamp,year_movie,genre
0,1,Toy Story,3.0,4.0,1999-12-11 13:36:47,1995,Adventure
1,1,Toy Story,6.0,5.0,1997-03-13 17:50:52,1995,Adventure
2,1,Toy Story,8.0,4.0,1996-06-05 13:37:51,1995,Adventure
3,1,Toy Story,10.0,4.0,1999-11-25 02:44:47,1995,Adventure
4,1,Toy Story,11.0,4.5,2009-01-02 01:13:41,1995,Adventure


* jika film memiliki lebih dari 1 genre maka setiap genre akan dipisahkan oleh garis "|" 

### Merubah format tanggal

In [7]:
df["timestamp"] = pd.to_datetime(df["timestamp"], format='%Y-%m-%d')
df["year"] = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day
df.head()

,movieId,title,userId,rating,timestamp,year_movie,genre,year,month,day
0,1,Toy Story,3.0,4.0,1999-12-11 13:36:47,1995,Adventure,1999.0,12.0,11.0
1,1,Toy Story,6.0,5.0,1997-03-13 17:50:52,1995,Adventure,1997.0,3.0,13.0
2,1,Toy Story,8.0,4.0,1996-06-05 13:37:51,1995,Adventure,1996.0,6.0,5.0
3,1,Toy Story,10.0,4.0,1999-11-25 02:44:47,1995,Adventure,1999.0,11.0,25.0
4,1,Toy Story,11.0,4.5,2009-01-02 01:13:41,1995,Adventure,2009.0,1.0,2.0


* pada dataset rating terdapat timestamp namun format seperti ini tidak mendukung pemrosesan data sehingga saya merubah format timestamp menjadi format per tahun, bulan dan tanggal.

### Memilih user secara random

In [8]:
user_id = int(pd.Series(user_movie_df.index).sample(1).values)
user_id

72088

* langkah berikutnya menentukan user secara random dengan cara mengambil sampel user secara random berdasarkan user id lalu didapatkan user id sekian

### Menentukan film user yang telah dipilih secara random diatas 

In [9]:
user_df = user_movie_df[user_movie_df.index == user_id]

movies_watched = user_df.columns[user_df.notna().any()].tolist()
len(movies_watched)

33

* Setelah mendapatkan user id secara random lalu menentukan film dan menghitung total film yang telah ditonton oleh user tersebut

### Mengakses data dan ID user yang melihat film yang sama

In [10]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()
movies_watched_df.shape

(138493, 33)

* Terdapat sekian user melihat film yang sama, setidaknya satu film dilihat oleh user yang dipilih secara random diatas

### Melihat seberapa banyak film yang dilihat oleh user

In [11]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head()

userId
1.0    4
2.0    4
3.0    8
4.0    1
5.0    0
dtype: int64

* menghitung setiap film yang telah ditonton oleh berapa user tidak termasuk null atau yang kosong

### Menghitung jumlah film yang telah dilihat

In [12]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
m_count = movies_watched_df.shape[1] 

### Memilih rasio 0.60 ID user yang telah menonton setidaknya 60% dari film

In [13]:
users_same_movies=user_movie_count[user_movie_count["movie_count"]/m_count > 0.6].sort_values("movie_count",ascending=False)
users_same_movies.nunique()

userId         408
movie_count     14
dtype: int64

* Menghitung jumlah film yang telah dilihat, lalu tentukan rasio sebesar 60% atau 0,6. sehingga user telah menonton setidaknya 60% dari film yang akan diambil.
* maka dari itu setidaknya 1 film telah dilihat oleh user yang begitu banyak maka dikerucutkan sebesar 60% dari data film yang telah dilihat.

In [14]:
users_same_movies.head(5)

,userId,movie_count
72087,72088.0,33
8404,8405.0,33
74141,74142.0,32
76629,76630.0,31
12130,12131.0,31


* kemudian dapat dilihat user id yang telah menonton film sebanyak 60% dari total film pada (poin 10)

### Menentukan user yang berkorelasi dengan user yang akan direkomendasikan dengan tidak mengabaikan rasio diatas (60%)

In [15]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)],
                      user_df[movies_watched]])
final_df.head()

title,*batteries not included,13 Going on 30,"13th Warrior, The",1408,2001: A Space Odyssey,2012,27 Dresses,28 Days,28 Days Later,28 Weeks Later,3 Ninjas,30 Days of Night,300,3:10 to Yuma,40 Days and 40 Nights,"40-Year-Old Virgin, The",Carrie,"Craft, The",Dangerous Minds,"Fly, The","Gods Must Be Crazy, The","League of Their Own, A","Negotiator, The","NeverEnding Story, The",Powder,"Road Warrior, The (Mad Max 2)",Rush Hour,Space Jam,Star Trek: The Motion Picture,Superman II,Time Bandits,Tron,Twelve Monkeys (a.k.a. 12 Monkeys)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1.0,NaN
981.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1506.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1848.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


* jadi user yang hanya menonton 1 film tidak termasuk rasio 60% keatas. dapat dilihat setiap user yang memiliki rasio minimal 60% telah menonton film ditampilkan disini mulai dari user id, rating dan judul film.

### Menghitung korelasi antar user

In [16]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,corr
0,72088.0,106067.0,-1.0
1,10677.0,51226.0,-1.0
2,52008.0,25408.0,-1.0
3,32860.0,91066.0,-1.0
4,108222.0,77695.0,-1.0


* pada proses ini akan menentukan korelasi antar user. user 1 dan user 2 akan di bandingkan dan dicari korelasi hingga mendapatkan korelasi antar pengguna terkait

### Mengambil dan Menganalisis korelasi antar user dengan memasukan nilai

In [17]:
top_users = corr_df[(corr_df["user_id_1"] == user_id) & (corr_df["corr"] >= 0.60)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)

top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.T

,7,6,5,4,3,2,1,0
userId,69682.000000,122435.000000,22549.000000,76154.000000,42885.000000,103253.000000,69377.000000,15616.000000
corr,0.991241,0.934262,0.917663,0.880705,0.857493,0.754386,0.671625,0.618397


* Setelah mendapatkan korelasi antar pengguna, kemudian dicari korelasi tertinggi dan dianalisis dengan memasukan nilai. namun pada user id 1 adalah user id sampel yang dipilih secara random diatas  nantinya akan direkomendasikan dengan user yang memiliki korelasi diatas 60%. 

### Menghitung bobot rating

In [18]:
rating = pd.read_csv('rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] = top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
0,69682.0,4.0,10,4.0,4.0
1,69682.0,5.0,32,5.0,5.0
2,69682.0,5.0,260,5.0,5.0
3,69682.0,5.0,316,5.0,5.0
4,69682.0,5.0,329,5.0,5.0


* Rating disini diambil dari dataset kemudian dihitung weighted rating berdasarkan korelasi dikali rating 

### Menghitung bobot rata_rata skor rekomendasi dan menampilkan film teratas

In [19]:
temp = top_users_ratings.groupby('movieId').sum()[['corr', 'weighted_rating']]
temp.columns = ['sum_corr', 'sum_weighted_rating']
temp.head()

,sum_corr,sum_weighted_rating
movieId,,
1,7.0,7.0
2,10.0,10.0
3,2.0,2.0
6,8.0,8.0
10,13.0,13.0


* jadi disini dihitung film teratas dari unique rating untuk movieId

### Menghitung bobot rata_rata skor rekomendasi dan menampilkan film teratas berdasarkan skor

In [20]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted_average_recommendation_score'] = temp['sum_weighted_rating']/temp['sum_corr']
recommendation_df['movieId'] = temp.index
recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)
recommendation_df.head(5)

,weighted_average_recommendation_score,movieId
movieId,,
1,1.0,1
6711,1.0,6711
6874,1.0,6874
6872,1.0,6872
6863,1.0,6863


* proses ini menghasilkan 5 skor tertinggi lalu ditampilkan pada tabel diatas

### merekomendasikan film yeng mungkin disukai user

In [21]:
movie = pd.read_csv('movie.csv')
movie.loc[movie['movieId'].isin(recommendation_df.head(5)['movieId'])]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
6601,6711,Lost in Translation (2003),Comedy|Drama|Romance
6753,6863,School of Rock (2003),Comedy|Musical
6762,6872,"House of the Dead, The (2003)",Action|Horror
6764,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller


* terdapat film yang disukai oleh user terkait, kemungkinan film yang belum di lihat dapat direkomendasikan kepada user

### Rating film dengan pengguna terkait

In [22]:
user_df = df.loc[df["userId"]==user_id]
user_df.head()

,movieId,title,userId,rating,timestamp,year_movie,genre,year,month,day
313954,24,Powder,72088.0,5.0,2012-06-06 19:58:04,1995,Drama,2012.0,6.0,6.0
361566,31,Dangerous Minds,72088.0,2.5,2012-06-06 19:57:13,1995,Drama,2012.0,6.0,6.0
389389,32,Twelve Monkeys (a.k.a. 12 Monkeys),72088.0,3.5,2012-06-06 20:00:33,1995,Mystery,2012.0,6.0,6.0
4379316,673,Space Jam,72088.0,1.0,2012-06-06 19:56:48,1996,Adventure,2012.0,6.0,6.0
4459193,724,"Craft, The",72088.0,2.5,2012-06-06 19:57:37,1996,Drama,2012.0,6.0,6.0


* jadi rating yang telah dimasukan user akan ditampilkan kedalam urutan berikut

### Rekomendasi akhir sesuai dengan judul film pengguna yang terakhir ditonton dan memiliki rating tertinggi 

In [23]:
user_df.groupby("title").agg({"rating": "max", "timestamp": "max"}).sort_values("timestamp", ascending=False)

,rating,timestamp
title,,
"40-Year-Old Virgin, The",3.0,2012-06-06 20:04:34
40 Days and 40 Nights,3.5,2012-06-06 20:04:26
3:10 to Yuma,5.0,2012-06-06 20:04:16
300,4.0,2012-06-06 20:03:54
30 Days of Night,3.0,2012-06-06 20:03:46
30 Minutes or Less,2.0,2012-06-06 20:03:40
30 Days of Night: Dark Days,3.5,2012-06-06 20:03:36
3 Ninjas,2.5,2012-06-06 20:03:28
28 Weeks Later,3.0,2012-06-06 20:03:10


* kemudian dibuat rekomendasi film user yang memiliki rating tertinggi berdasarkan timestamp. selanjutnya diurutkan berdasarkan film yang terakhir kali ditonton oleh user.